In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
df_train.columns

In [ ]:
df_train.drop([ 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis = 1,inplace = True)

In [ ]:
df_train

In [ ]:
df_train['Id'] = df_train['Id'].apply(lambda x: x + '.jpg')
#df_train.Pawpularity = df_train.Pawpularity.astype('str')

In [ ]:
df_train

In [ ]:
base_model_weight = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1.0/255.0,
                horizontal_flip = True,
                fill_mode = "nearest", zoom_range = 0.2,
                width_shift_range = 0.2, height_shift_range=0.2,
                rotation_range=30,
)

In [ ]:
train_data = train_datagen.flow_from_dataframe(
            dataframe = df_train,
            directory='../input/petfinder-pawpularity-score/train',
            x_col = "Id",
            y_col = "Pawpularity",
            has_ext=True,
            class_mode="raw",
            batch_size = 128,
            target_size=(128,128),
            shuffle = True,
        )


In [ ]:
input_layer = tf.keras.layers.Input(shape =(128,128,3))
base_model = tf.keras.applications.InceptionV3(weights=None, include_top=False, input_tensor=input_layer)
base_model.load_weights(base_model_weight)

In [ ]:
len(base_model.layers)

In [ ]:
x = base_model.output
head_model = tf.keras.layers.BatchNormalization()(x)
head_model = tf.keras.layers.GlobalAveragePooling2D()(head_model)
head_model = tf.keras.layers.Dense(1024,activation = 'relu')(head_model)
predictions =  tf.keras.layers.Dense(1)(head_model)
network = tf.keras.Model(inputs = base_model.input,outputs = predictions)

In [ ]:
for layer in base_model.layers[:280]:
    layer.trainable = False

for layer in network.layers:
    layer.trainable = True

In [ ]:
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = tf.keras.optimizers.SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)
network.compile(optimizer='adam', loss = 'mean_absolute_error',metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()])

In [ ]:
eraly_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)

In [ ]:
NUB_TRAIN_STEPS = train_data.n // train_data.batch_size

In [ ]:
history = network.fit(  train_data,
                         steps_per_epoch=NUB_TRAIN_STEPS,
                         epochs=epochs, 
                         callbacks=[eraly_stop, reduce_lr],
                         verbose=1
                     )

In [ ]:
test_datagen = ImageDataGenerator(
                rescale = 1.0/255.0,
)
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
df_test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis = 1,inplace = True)

In [ ]:
df_test.Id = df_test.Id.apply(lambda x:x +'.jpg')

In [ ]:
df_test.head()

In [ ]:
test_data  = test_datagen.flow_from_dataframe(
            dataframe = df_test,
            directory='../input/petfinder-pawpularity-score/test',
            x_col = "Id",
            shuffle=False,
            class_mode=None,
            target_size=(128,128),
        )

In [ ]:
pred = network.predict(test_data)

In [ ]:
pred

In [ ]:
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
sub.head()

In [ ]:
sub['Pawpularity'] = pred

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv',index = False)